In [2]:
import torch
from transformers import BertTokenizer, BertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model.config.id2label = {0: "location", 1: "people name"}
text = "My name is Aman. I am a student."
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)


In [15]:
with torch.no_grad():
    outputs = model(**inputs)

predicted_labels_indices = torch.argmax(outputs.logits, dim=2)


In [19]:
# Convert token IDs to labels
label_names = [model.config.id2label[label_id.item()] for label_id in predicted_labels_indices]

# Print token and corresponding label name
decoded_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
for token, label_name in zip(decoded_tokens, label_names):
    if '##' in token:
        # Handle subword tokens by appending them to the previous token
        print(f"Token: {decoded_tokens[-1]}{token.replace('##', '')}\tPredicted Label: {label_name}")
    else:
        print(f"Token: {token}\tPredicted Label: {label_name}")

RuntimeError: a Tensor with 12 elements cannot be converted to Scalar

In [34]:
from transformers import BertTokenizer, BertForTokenClassification

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased')

# Modify the id2label mapping in the model's configuration
model.config.id2label = {0: "location", 1: "people name",2:"organization"}  # Modify this mapping according to your labels

# Define an example input in Nepali
text = "मेरो नाम अमन हो। म एक विद्यार्थी हूँ।"

# Tokenize and encode input
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

inputs
# # Run inference
# with torch.no_grad():
#     outputs = model(**inputs)

# # Get predicted labels (integer indices)
# predicted_label_indices = torch.argmax(outputs.logits, dim=2)[0]

# # Convert label indices to label names using modified model's configuration
# label_names = [model.config.id2label[label_id.item()] for label_id in predicted_label_indices]

# # Print token and corresponding label name
# decoded_tokens = []
# for token, label_name in zip(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]), label_names):
#     if '##' in token and len(decoded_tokens) > 0:
#         decoded_tokens[-1] += token.replace('##', '')
#     else:
#         decoded_tokens.append(token.replace('##', ''))
    
# for token, label_name in zip(decoded_tokens, label_names):
#     print(f"Token: {token}\tPredicted Label: {label_name}")


c:\Users\Rabin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[   101,  40265, 101841,  21645,    851,  62752,  13220,    920,    889,
          11186,  55190,  97110,  19885,  69002,  10914,    899,  15778,  28462,
            920,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

## First Phase

In [48]:
# Loading and preprocessing labelled data
import pandas as pd

# csv data load gareko
data = pd.read_csv('chunk_2.csv')

# text ra label chuttako
texts=data['Column1'].tolist()
labels=data['Column2'].tolist()

# labels lai index diyeko
labels_to_index = {'O': 0,'B-Person': 1,'B-Date': 2,'B-Organization': 3,'B-Location': 4}
missing_label = 'O'  # Update this based on your use case

label_indices = []

for sequence_labels in labels:
    if isinstance(sequence_labels, str):
        label_indices.append([labels_to_index.get(label, labels_to_index[missing_label]) for label in sequence_labels])
    else:
        # Handle non-string or missing values as needed
        # For example, replace with a default label index or skip the sequence
        label_indices.append([labels_to_index[missing_label]])


In [52]:
# tokenization and data preparation
from torch.utils.data import Dataset, DataLoader

# tokenizing and encoding text
tokenized_texts = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Defining Custom pytorch dataset
class custom_datasets(Dataset):
    def __init__(self, tokenized_texts, label_indices):
        self.tokenized_texts = tokenized_texts
        self.label_indices = label_indices
    
    def __len__(self):
        return len(self.tokenized_texts['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.tokenized_texts['input_ids'][idx],
            'attention_mask': self.tokenized_texts['attention_mask'][idx],
            'labels': self.label_indices[idx]
        }

# Create a DataLoader

label_tensors = [torch.tensor(indices) for indices in label_indices]
batch_size = 32
train_dataset = custom_datasets(tokenized_texts, label_tensors)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [51]:
# Fine tuning 
from transformers import BertForTokenClassification, AdamW
import torch

# Load pre-trained BERT model for token classification
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(labels_to_index))

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
for batch in train_dataloader:
    optimizer.zero_grad()
    inputs = {
        'input_ids': batch['input_ids'],
        'attention_mask': batch['attention_mask'],
        'labels': batch['labels']  # Use the tensor of label indices
    }
    outputs = model(**inputs)
    loss = outputs.loss
    total_loss += loss.item()
    loss.backward()
    optimizer.step()
    
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss:.4f}')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Rabin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AttributeError: 'list' object has no attribute 'view'